In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [48]:

train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")


In [49]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [50]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [51]:
print(train_df.shape)
print(test_df.shape)
train_df.columns

(891, 12)
(418, 11)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [52]:
input_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'
print(train_df[target].isnull().sum())

0


In [53]:
for col in input_cols:
    print(col, train_df[col].isnull().sum(), train_df[col].dtype)


Pclass 0 int64
Sex 0 object
Age 177 float64
SibSp 0 int64
Parch 0 int64
Fare 0 float64
Embarked 2 object


In [54]:
cat_cols = ['Sex', 'Embarked']

In [55]:
encoder = {}
missing_values = {}

for col in cat_cols:
    missing_values[col] = train_df[col].mode()[0]
    train_df[col] = train_df[col].fillna(missing_values[col])
    encode = LabelEncoder().fit(train_df[col])
    encoder[col] = encode
    train_df[col] = encode.transform(train_df[col])

In [56]:
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2
5,6,0,3,"Moran, Mr. James",1,NaN,0,0,330877,8.4583,NaN,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,E46,2
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.0,3,1,349909,21.0750,NaN,2
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.0,0,2,347742,11.1333,NaN,2
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.0,1,0,237736,30.0708,NaN,0


In [57]:
missing_values

{'Sex': 'male', 'Embarked': 'S'}

In [59]:
num_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
for col in num_cols:
    missing_values[col] = train_df[col].median()
    train_df[col] = train_df[col].fillna(missing_values[col])

# missing_values['Age'] = train_df['Age'].median()
# train_df['Age'] = train_df['Age'].fillna(missing_values['Age'])
# missing_values['Age']

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

# model = LogisticRegression()
model = RandomForestClassifier()
# model = HistGradientBoostingClassifier()

In [61]:

# model = DecisionTreeClassifier()
model.fit(train_df[input_cols], train_df[target])

RandomForestClassifier()

In [62]:
pred = model.predict(train_df[input_cols])
print('Accuracy Score:', accuracy_score(train_df[target], pred))

Accuracy Score: 0.9797979797979798


In [63]:
for col in input_cols:
    if col in missing_values:
        test_df[col] = test_df[col].fillna(missing_values[col])


In [64]:

for col in cat_cols:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(missing_values[col])
        test_df[col] = test_df[col].map(lambda s: '<unknown>' if s not in encoder[col].classes_ else s)
        if '<unknown>' not in encoder[col].classes_:
            encoder[col].classes_ = np.append(encoder[col].classes_, '<unknown>')
        test_df[col] = encoder[col].transform(test_df[col])

In [66]:
pred_test = model.predict(test_df[input_cols])

submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': pred_test
})
submission.to_csv('Random.csv', index=False)